<a href="https://colab.research.google.com/github/doubleblindreview2/jbr_video_mining/blob/master/video_mining_yt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **JBR Video Mining Script**
- **Output**: This script extracts various video features from a given set of videos and stores these features in multiple .csv files, which can be opened with Excel or any text editor afterwards
- **Input**: You need to provide a Google Drive Account with a folder with video files
- **Execution**: In order to execute this script, run all cells from top to bottom and follow the instructions

Before you start:  **<font color='red'>Please click -> Runtime -> Change runtime type -> GPU in top menu</font>**

In [0]:
name = 'amos' # change to jochen or jasper
num = 0       # change to 1 for second and 2 for third notebook 

## **1. Download Models**
_You need a Google Account to verify your legitimate use; The models will be automatically downloaded from a public GDrive_
_____________________________________________________________________________



In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from tqdm import tqdm

Running the next cell will create a link. **You need to click this link and enter your Google Account credentials. Copy the code and enter it in the space below.**

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Running the next cell will download required models from a public GDrive. After running it, you can verify success by clicking on the folder icon on the left side of this screen.There should be a 'downloads' folder containing various files (You may need to click 'Refresh').

In [0]:
if not os.path.exists('/content/downloads/'): os.makedirs('/content/downloads/')
os.chdir('./downloads/')

folder_id = '1e-UQc-ylzVOOvW2ZiOCpnP-EEziHA4cQ'
file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(folder_id)}).GetList()
for file in tqdm(file_list):
    file.GetContentFile(file['title'])

os.chdir('/content/')

## **2. Provide Input & Select Features**
_You need to provide a Google Drive Account, which includes a folder with video files for video mining._
_____________________________________________________________


Running the next cell will create a link. **You need to click this link and enter your Google Account credentials. Copy the code and enter it in the space below.**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

## **2.1 Get YT Vids**
_Simply run all cells._

In [0]:
import time
import os
import shutil
import pandas as pd

In [0]:
!pip install -q youtube-dl

In [0]:
if not os.path.exists('/content/yt_vids/'): 
  os.mkdir('/content/yt_vids/')

In [0]:
yt_ids = pd.read_csv(f'/content/drive/My Drive/trailer/vids/batch_{name}_{num}.csv',index_col = 0)

In [0]:
extract_length           = False # get length of video
extract_cuts             = False # get scene cuts
extract_colors           = False # get brightness and  color information
extract_faces            = False # get faces
extract_emotions         = True # get 8 different emotions per face 
extract_objects          = False # get 80 objects
extract_variance         = True # get semantic variance
extract_quality          = False # get quality

in_folder  = '/content/yt_vids/'  # folder with videos, names are used as IDs
out_folder = '/content/drive/My Drive/trailer/preds/' # folder to store extracted features

device = 0

In [0]:
for YOUTUBE_ID in list(yt_ids.yt_id):
  if not YOUTUBE_ID+'.mp4' in os.listdir('/content/yt_vids/'):
    !youtube-dl -f 'bestvideo[ext=mp4]' --output "/content/yt_vids/"$YOUTUBE_ID".%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID

log_name   = f'2020-05-23_logfile_{start}.csv' # name of lofile, pls include .csv ending 
log_folder = '/content/drive/My Drive/trailer/logs/' # folder for logfile

start_index = 0
end_index = len(os.listdir(in_folder))

# device number of GPU, do not change and set Runtime Type to GPU
variables = ''
for variable in ['extract_length',
                'extract_cuts',
                'extract_colors',
                'extract_faces',
                'extract_emotions',
                'extract_variance',
                'extract_objects',
                'extract_quality',
                'start_index',
                'end_index',
                'device']:
  variables += ' --' + variable.replace('_','-') + ' ' + str(eval(variable))

for variable in ['in_folder',
                'out_folder',
                'log_folder',
                'log_name']:
  variables += ' --' + variable.replace('_','-') + ' "' + str(eval(variable))+'"'

!python "/content/downloads/jbr_run.py" {variables}

# shutil.rmtree('/content/scene_imgs')
# for i in os.listdir('/content/yt_vids'):
#   !rm -rf "/content/yt_vids/"$i

# **<font color='red'>Thanks, that's it!!!</font>**

**Please provide the correct pathes to your folder with videos. You probably need to change the text after '/My Drive/**.

You can verify the correct path by clicking on the folder icon on the left side of this screen and search within the 'drive' folder. (You may need to click 'Refresh') 

In [0]:
in_folder  = '/content/drive/My Drive/trailer/vids/'  # folder with videos, names are used as IDs

You **can unselect features** you are not interested in **by writing 'False' instead of 'True'**. This may drastically increase processing speed.

In [0]:
extract_length           = False # get length of video
extract_cuts             = False # get scene cuts
extract_colors           = False # get brightness and  color information
extract_faces            = False # get faces
extract_emotions         = True # get 8 different emotions per face 
extract_objects          = False # get 80 objects
extract_variance         = True # get semantic variance
extract_quality          = False # get quality

## **4. Provide Optional Input**
_**Run all cells - no additional input required, however you may change things to customize execution**_
_____________________________________________________________

The following folders and the logfile will be created to store the results of your analysis. **You do not have to change them, unless you want to use your custom folder** structure to store results

In [0]:
out_folder = '/content/drive/My Drive/trailer/preds/' # folder to store extracted features
log_name   = '2020-05-23_logfile.csv' # name of lofile, pls include .csv ending 
log_folder = '/content/drive/My Drive/trailer/logs/' # folder for logfile

Setting a start and end index allows you to only analyze a subset of the videos in your folder. **You do not have to change this, if you want to analyze all videos in your provided folder.**

In [0]:
start_index = 0
end_index = len(os.listdir(in_folder))

The following code tells the code to use the GPU if required.  **You do not have to change this.**

In [0]:
device = 0

## **4. Extract Selected Features**
_**Run all cells - no additional input required**_

_This will extract your selected features by looping through your provided video files folder. (This may take siginificant amount of time)._

In [0]:
 # device number of GPU, do not change and set Runtime Type to GPU
variables = ''
for variable in ['extract_length',
                 'extract_cuts',
                 'extract_colors',
                 'extract_faces',
                 'extract_emotions',
                 'extract_variance',
                 'extract_objects',
                 'extract_quality',
                 'start_index',
                 'end_index',
                 'device']:
  variables += ' --' + variable.replace('_','-') + ' ' + str(eval(variable))

for variable in ['in_folder',
                 'out_folder',
                 'log_folder',
                 'log_name']:
  variables += ' --' + variable.replace('_','-') + ' "' + str(eval(variable))+'"'

In [0]:
variables

In [0]:
!python "/content/downloads/jbr_run.py" {variables}

# ONLY TESTS


In [0]:
import argparse
import os
import sys
import subprocess
from zipfile import ZipFile
import sys
import csv
import numpy as np
import pandas as pd
from moviepy.video.io.VideoFileClip import VideoFileClip
import cv2
from os.path import isfile, join
import datetime as dt
from datetime import date
from datetime import datetime
from google.colab import drive
from google.colab.patches import cv2_imshow
from tqdm import tqdm
import re
from keras.preprocessing import image
from keras.models import model_from_json

In [0]:
  input_path = '/content/yt_vids/Xr_N3-kF2hk.mp4'
  out_path = '/content/yt_vids/4'
  name = 'Xr_N3-kF2hk'
  skip_frames=20


# def get_emotions(input_path, out_path, name, skip_frames=10):
  # read video file
  cap = cv2.VideoCapture(input_path)
  
  # load preconfigured model and precomputed weights
  model = model_from_json(open("/content/downloads/facial_expression_model_structure.json", "r").read())         ## REQUIRED TO CHANGE PATH
  model.load_weights('/content/downloads/facial_expression_model_weights.h5')                                    ## REQUIRED TO CHANGE PATH
  face_cascade = cv2.CascadeClassifier('/content/downloads/haarcascade_frontalface_default.xml')    ## REQUIRED TO CHANGE PATH
  
  df = pd.DataFrame(columns=['TimeStamp','Frame','Number of Faces (left->right)','Face Position','angry','disgust','fear','happy','sad','surprise','neutral'])
  
  try:
      i = 0
      while (cap.isOpened()):
          # cap.read() -> checks whether or not frame has been read correctly. Returns boolean value
          ret, frame = cap.read()
          if not (ret):
              break
  
          # for every x frame
          if cap.get(cv2.CAP_PROP_POS_FRAMES) % skip_frames == 0:
              gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
              faces = face_cascade.detectMultiScale(gray, 1.3, 5)
  
              face_counter = 0
              
              for (x, y, w, h) in faces:
                  # cuts frames into required data (requires 4-Dim instead of 3, therefore i used the given approach from the project)
                  cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                  detected_face = frame[int(y):int(y + h), int(x):int(x + w)]  # crop detected face
                  detected_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2GRAY)  # transform to gray scale
                  detected_face = cv2.resize(detected_face, (48, 48))  # resize to 48x48
                  img_pixels = image.img_to_array(detected_face)
                  img_pixels = np.expand_dims(img_pixels, axis=0)/255
  
                  # prediction
                  predictions = model.predict(img_pixels)

                  df.loc[i,'TimeStamp'] = cap.get(cv2.CAP_PROP_POS_MSEC)
                  df.loc[i,'Frame'] = cap.get(cv2.CAP_PROP_POS_FRAMES)
                  df.loc[i,'Number of Faces (left->right)'] = len(faces)
                  df.loc[i,'Face Position'] = face_counter
                  df.loc[i,'angry':] = pd.Series(predictions[0],index=df.loc[:,'angry':].columns)

                  face_counter += 1
                  i +=1

      # When everything done, release the capture
      cap.release()
      cv2.destroyAllWindows

      df.to_csv(out_path + name + '_FrameLevel_emotions.csv')
      pd.concat([df.loc[:,'angry':].mean(),df.loc[:,'angry':].max()],axis=1).rename(columns={0:'Average',1:'Max'}).to_csv(out_path + name + '_VideoLevel_emotions.csv')
  
      print(df.head())
  except:
      print(sys.exc_info()[1])


In [0]:
df = get_emotions('/content/yt_vids/10r9ozshGVE.mp4','/content/drive/My Drive/trailer/preds/5MBjAN7jqsQ/','5MBjAN7jqsQ')

In [0]:
df.head()
pd.DataFrame(df.loc[:,'angry':].mean(),columns=['Average'])
vl = pd.concat([df.loc[:,'angry':].mean(),df.loc[:,'angry':].max()],axis=1).rename(columns={0:'Average',1:'Max'})
# vl.columns=['Average','Max']
vl
# df.loc[:,'angry':].max()


In [0]:
a = 100
a/=10
print(a)

In [0]:
def get_emotions(input_path, out_path, name, skip_frames=10):
  # read video file
  cap = cv2.VideoCapture(input_path)
  
  # load preconfigured model and precomputed weights
  model = model_from_json(open("/content/downloads/facial_expression_model_structure.json", "r").read())         ## REQUIRED TO CHANGE PATH
  model.load_weights('/content/downloads/facial_expression_model_weights.h5')                                    ## REQUIRED TO CHANGE PATH
  face_cascade = cv2.CascadeClassifier('/content/downloads/haarcascade_frontalface_default.xml')    ## REQUIRED TO CHANGE PATH
  
  df = pd.DataFrame(columns=['TimeStamp','Frame','Number of Faces (left->right)','Face Position','angry','disgust','fear','happy','sad','surprise','neutral'])
  
  try:
      i = 0
      while (cap.isOpened()):
          # cap.read() -> checks whether or not frame has been read correctly. Returns boolean value
          ret, frame = cap.read()
          if not (ret):
              break
  
          # for every x frame
          if cap.get(cv2.CAP_PROP_POS_FRAMES) % skip_frames == 0:
              gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
              faces = face_cascade.detectMultiScale(gray, 1.3, 5)
  
              face_counter = 0
              
              for (x, y, w, h) in faces:
                  # cuts frames into required data (requires 4-Dim instead of 3, therefore i used the given approach from the project)
                  cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                  detected_face = frame[int(y):int(y + h), int(x):int(x + w)]  # crop detected face
                  detected_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2GRAY)  # transform to gray scale
                  detected_face = cv2.resize(detected_face, (48, 48))  # resize to 48x48
                  img_pixels = image.img_to_array(detected_face)
                  img_pixels = np.expand_dims(img_pixels, axis=0)/255
  
                  # prediction
                  predictions = model.predict(img_pixels)

                  df.loc[i,'TimeStamp'] = cap.get(cv2.CAP_PROP_POS_MSEC)
                  df.loc[i,'Frame'] = cap.get(cv2.CAP_PROP_POS_FRAMES)
                  df.loc[i,'Number of Faces (left->right)'] = len(faces)
                  df.loc[i,'Face Position'] = face_counter
                  df.loc[i,'angry':] = pd.Series(predictions[0],index=df.loc[:,'angry':].columns)

                  face_counter += 1
                  i +=1

      # When everything done, release the capture
      cap.release()
      cv2.destroyAllWindows

      df.to_csv(out_path + name + '_FrameLevel_emotions.csv')
      pd.concat([df.loc[:,'angry':].mean(),df.loc[:,'angry':].max()],axis=1).rename(columns={0:'Average',1:'Max'}).to_csv(out_path + name + 'VideoLevel_emotions.csv')
  
      return df
  except:
      return sys.exc_info()[1]